In [ ]:
import sqlite3 as db
import pandas as pd
import re


# List of common suffixes to remove
suffixes = ['gmbh', 'co', 'kg', 'inc', 'llc', 'ltd', 'ag', 'corporation', 'corp','deutschland','raffinerie','oel','werk','nord','sud','europa','holding','europe','se','oil','aluminium','leuna','trebsen']


## 1. Load databases and extract key words from company names for further comparison

In [ ]:
#load heat production database
# If the first row contains headers, you can skip it using the 'header' parameter:
df_ref = pd.read_excel('points.xlsx')

# Ensure all columns are converted to strings before concatenation
df_ref['StreetNameAndNumber'] = df_ref['StreetNameAndNumber'].astype(str)
df_ref['PostalCode'] = df_ref['PostalCode'].astype(str)

# Perform the concatenation
df_ref['Address'] = df_ref['StreetNameAndNumber'] + ',' + df_ref['PostalCode'] + ', Deutschland'

df_ref = df_ref.drop(columns=['Unnamed: 0','SiteId','ID_Eurostat','ProcessInfo','NUTS1ID','NUTS3ID','geometry','StreetNameAndNumber','Country'])

print(len(df_ref))
df_ref.head()

##### Selecting main words from company names to search for them in the Handelsregister database

In [ ]:
df = df_ref

# Function to clean company names
def clean_name(name):
    # Remove common suffixes and extra whitespace
    pattern = r'\b(?:' + '|'.join(suffixes) + r')\b'
    # Remove special characters (e.g., & . ,)
    name = re.sub(r'[&.,+()]', '', name)
    
    return re.sub(pattern, '', name, flags=re.IGNORECASE).strip()

# Apply the clean_name function to create a new column for comparison
df['CleanedName'] = df['CompanyName'].apply(clean_name)

print("length df: ", len(df))

In [ ]:
#load handelsregister
conn = db.connect('latlongsdata.db')
df_h = pd.read_sql_query('select * from Lat_Long_Table_HandelsregisterV3', conn)
#df_h = pd.read_sql_query('select * from Lat_Long_Table_Handelsregister_Referencepaper', conn)
conn.close()
df_h = df_h.drop_duplicates(subset=['name','register_identifier','zip'], keep='first')
print(len(df_h))
#df_h = df_h.drop(columns=['level_0'])
df_h.head()

In [ ]:
df2 = df_h

# Function to clean company names
def clean_name(name):
    # Remove common suffixes and extra whitespace
    pattern = r'\b(?:' + '|'.join(suffixes) + r')\b'
    # Remove special characters (e.g., & . ,)
    name = re.sub(r'[&.,+()]', '', name)
    
    return re.sub(pattern, '', name, flags=re.IGNORECASE).strip()

# Apply the clean_name function to create a new column for comparison
df2['CleanedName'] = df2['name'].apply(clean_name)

print("length df: ", len(df2))

## 2. Finding coincidences based on comparing company names' key words and then postal codes

### A. Compare names and then postal codes

In [ ]:
df1 = df

df2 = df2

# Create an empty list to store merged rows
merged_rows = []

# Iterate through each row in df1
for i, row1 in df1.iterrows():
    words1 = set(row1['CleanedName'].split())
    
    # Compare with each row in df2
    for j, row2 in df2.iterrows():
        words2 = set(row2['CleanedName'].split())
        
        # If there's at least one common word, merge the rows
        if words1.intersection(words2):
            merged_row = {**row1, **row2}  # Merge the two rows into one dictionary
            merged_rows.append(merged_row)  # Add the merged row to the list

# Convert the list of merged rows into a new DataFrame
merged_df = pd.DataFrame(merged_rows)

In [ ]:
#Keep rows where postalcodes coincide
filtered_df = merged_df[merged_df['zip'] == merged_df['PostalCode']]

### B. Comparing postal codes and then names

In [ ]:
#comparing postal code
# Merge the two dataframes on the Postal code column
filtered_df2 = pd.merge(df_h, df_ref, left_on='zip', right_on='PostalCode', how='inner')

In [ ]:
merged_df = filtered_df2
# Function to clean and split text into words
def clean_and_split(text):
    # Remove special characters and split into words
    words = re.sub(r'[^\w\s]', '', text).lower().split()
    return [word for word in words if word not in suffixes]

# Apply the function to both columns
merged_df['name_words'] = merged_df['name'].apply(clean_and_split)
merged_df['CompanyName_words'] = merged_df['CompanyName'].apply(clean_and_split)

# Function to check if there is any common word between two lists
def has_common_word(list1, list2):
    return any(word in list2 for word in list1)

# Filter the DataFrame
filteredmerged_df = merged_df[merged_df.apply(lambda row: has_common_word(row['name_words'], row['CompanyName_words']), axis=1)]

# Drop the helper columns if needed
filteredmerged_df = filteredmerged_df.drop(columns=['name_words', 'CompanyName_words'])
filteredmerged_df.drop_duplicates()

## C. Now we merge both (1filtered_df and 2filteredmerged_df)

In [ ]:
filteredmerged_df = filteredmerged_df.drop(columns=['CleanedName_x', 'CleanedName_y'])

# Concatenate the two DataFrames
merged_dfh2pot = pd.concat([filteredmerged_df, filtered_df])

# Drop duplicate rows
# Optionally, you can specify which columns to consider for detecting duplicates
merged_dfh2pot.drop_duplicates(subset=['H2-Potential in TWh', 'CompanyName'], keep='first')
merged_dfh2pot = merged_dfh2pot.drop_duplicates()
len(merged_dfh2pot)
finaldf = merged_dfh2pot

## 3. Check from which documents I can get information from the XML files available in the Handelsregister

In [ ]:
#antes de runear esto asegurarme de que todos los si docs nuevos están en la carpeta de si docs from other tries 
import os
df
df = merged_dfh2pot
df = df[['name','register_identifier']]

df = df.rename(columns={'name': 'Name', 'register_identifier': 'Reference_number'})

# Directory where the documents are stored
doc_directory = r"Directory_X"

# Lists to store company names and reference numbers with and without matching documents
matching_companies = []
matching_refnum = []
no_matching_companies = []
no_matching_refnum = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    company_name = row['Name']
    reference_number = str(row['Reference_number'])

    # Flag to check if a matching document was found
    found = False

    # Search term is simply the reference number
    search_term = reference_number

    for doc_name in os.listdir(doc_directory):
        if search_term in doc_name:
            matching_companies.append(company_name)
            matching_refnum.append(reference_number)
            found = True
            break  # Exit loop if a match is found

    if not found:
        #print(f"No document found for {company_name} with reference number {reference_number}")
        no_matching_companies.append(company_name)
        no_matching_refnum.append(reference_number)

print("Xml files found for ", len(matching_companies), "companies")
print("Xml files not found for ", len(no_matching_companies), "companies")

## 4. Extract GRUNDKAPITAL / STAMMKAPITAL / HAFTEINLAGE values and add them to the dataframe

In [ ]:
import os
import xml.etree.ElementTree as et
import pandas as pd

# Assuming matching_companies and matching_refnum are already defined
combined_data = list(zip(matching_companies, matching_refnum))

# Create DataFrame from combined data
df_grundkapital = pd.DataFrame(combined_data, columns=['Name', 'Reference_number'])
stammkapital = []
grundkapital = []
hafteinlage = []

# Directory containing XML files
download_directory = r"Directory_X"

# Define the namespaces
namespaces = {
    'xjustiz': 'http://www.xjustiz.de'
}

# Iterate over each company
for company in df_grundkapital.Name:
    # Get the reference number of the company
    reference_number = str(df_grundkapital[df_grundkapital['Name'] == company]['Reference_number'].iloc[0])
    
    xml_file_path = None
    for xml_file in os.listdir(download_directory):
        if xml_file.endswith(".xml") and reference_number in xml_file:
            xml_file_path = os.path.join(download_directory, xml_file)
            break  # Exit loop if a matching file is found

    if xml_file_path:
        with open(xml_file_path, "r", encoding="utf-8") as file:
            xml_content = file.read()
        
        xml_tree = et.fromstring(xml_content)

        # STAMMKAPITAL
        stammkapital_value = xml_tree.findall(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:kapitalgesellschaft/xjustiz:zusatzGmbH/xjustiz:stammkapital/xjustiz:zahl", namespaces)
        stammkapital.append(stammkapital_value[0].text if stammkapital_value else "0")

        # GRUNDKAPITAL
        grundkapital_value = xml_tree.findall(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:kapitalgesellschaft/xjustiz:zusatzAktiengesellschaft/xjustiz:grundkapital/xjustiz:hoehe/xjustiz:zahl", namespaces)
        grundkapital.append(grundkapital_value[0].text if grundkapital_value else "0")

        # HAFTEINLAGE - Extract all, sum them up, and append to the list
        hafteinlage_elements = xml_tree.findall(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:personengesellschaft/xjustiz:zusatzGmbH/xjustiz:datenKommanditist/xjustiz:hafteinlage/xjustiz:zahl", namespaces)
        if not hafteinlage_elements:
            hafteinlage_elements = xml_tree.findall(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:personengesellschaft/xjustiz:zusatzKG/xjustiz:datenKommanditist/xjustiz:hafteinlage/xjustiz:zahl", namespaces)
        
        total_hafteinlage = sum(float(el.text) for el in hafteinlage_elements)
        hafteinlage.append(str(total_hafteinlage) if hafteinlage_elements else "0")
    else:
        stammkapital.append("0")
        grundkapital.append("0")
        hafteinlage.append("0")

# Add the extracted data to the DataFrame
df_grundkapital['Stammkapital'] = stammkapital
df_grundkapital['Grundkapital'] = grundkapital
df_grundkapital['Hafteinlage'] = hafteinlage

# Display the final DataFrame
print(df_grundkapital)


In [ ]:
df_grundkapitaldd = df_grundkapital.drop_duplicates()
len(df_grundkapitaldd)

In [ ]:
# Adjust format of dataframe
df_grundkapital.rename(columns={'Reference_number': 'register_identifier'}, inplace=True)
combined_df = pd.merge(df_grundkapital, finaldf, on='register_identifier', how='inner')
combined_df = combined_df.drop(columns=['Name','CleanedName'])
combined_df = combined_df.drop_duplicates()
combined_df.head()

In [ ]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = db.connect('databse.db')

# Save the DataFrame to a table in the SQLite database
combined_df.to_sql('Hydrogen-Capital', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [ ]:
len(combined_df)

## 5. Save names for which results were not found for further searches

In [ ]:
filtered_df_ref = df_ref[~df_ref['CompanyName'].isin(combined_df['CompanyName'])]

filtered_df_ref = filtered_df_ref.drop(columns=['Subsector_Name','H2-Potential in TWh'])
filtered_df_ref = filtered_df_ref.assign(source='Heatpot')

In [ ]:
conn = db.connect('database.db')

# Save the DataFrame to a table in the SQLite database
filtered_df_ref.to_sql('NotMatched', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [ ]:
combined_data = list(zip(no_matching_companies, no_matching_refnum))
combined_data = pd.DataFrame(combined_data, columns=['Name', 'Reference_number'])

conn = db.connect('database.db')

# Save the DataFrame to a table in the SQLite database
combined_data.to_sql('NotMatched', conn, if_exists='replace', index=False)

# Close the connection
conn.close()


## 6. Cleaning the data

#### List of company names to delete: these have been found by manually checking the dataset

In [ ]:
df = combined_df

# Define the condition for the rows to be deleted
condition1 = (df['name'] == 'Märker Kalk GmbH') & (df['CompanyName'] == 'Märker Zementwerk GmbH')
# Delete the rows that meet the condition
df = df[~condition1]

condition2 = (df['name'] == 'Märker Zementwerk GmbH') & (df['CompanyName'] == 'Märker Kalk GmbH')
df = df[~condition2]

condition3 = (df['name'] == 'Märker Zement GmbH') & (df['CompanyName'] == 'Märker Kalk GmbH')
df = df[~condition3]


df_h2b = df

companies_to_delete = [
    "Katharina Tillmann Papier und Wellpappenfabrik e.K.",
    "ArcelorMittal Duisburg GmbH",
    "TRIMET ALUMINIUM SE, Niederlassung Hamburg",
    "Mayr-Melnhof Gernsbach GmbH"
]

# Filter the DataFrame to get the rows to delete
deleted_rows = df_h2b[df_h2b['CompanyName'].isin(companies_to_delete)]

# Filter the DataFrame to exclude the specified companies
df_filtered = df_h2b[~df_h2b['CompanyName'].isin(companies_to_delete)]

# Reset the index of the resulting DataFrame
df_filtered = df_filtered.reset_index(drop=True)

# Display the deleted rows
print("Deleted Rows:")
print(deleted_rows)

df_h2b = df_filtered

#### Merging administrative subsidiaries

In [ ]:
df = df_h2b

# Groups to combine
companies_merge = {
    'ArcelorMittal Duisburg GmbH': ['ArcelorMittal Duisburg Beteiligungsgesellschaft mbH'],
    'BASF Fuel Cell GmbH': ['BASF Fuel Cell Pensionsverwaltung GmbH'],
    'BASF SE': ['BASF VC Beteiligungs- und Managementgesellschaft mbH', 
                'BASF Trostberger Grundbesitz GmbH', 
                'BASF Ludwigshafen Grundbesitz SE & Co. KG', 
                'BASF Lizenz GmbH', 
                'BASF Jobmarkt GmbH', 
                'BASF Beteiligungsgesellschaft mbH', 
                'BASF Battery Technology Investment GmbH & Co. KG', 
                'BASF watertechnologies GmbH & Co. KG', 
                'BASF Biorenewable Beteiligungs GmbH & Co. KG'],
    'Panther Packaging GmbH & Co.KG': ['Beteiligungsgesellschaft Panther Packaging mbH'],
    'Covestro AG': ['Covestro Second Real Estate GmbH'],
    'DS Smith Paper Deutschland GmbH': ['DS Smith Transport Services GmbH'],
    'Dow Deutschland Inc., Zweigniederlassung Stade': ['Dow Deutschland Anlagengesellschaft mbH'],
    'Glatfelter Gernsbach GmbH': ['Glatfelter Services GmbH'],
    'INOVYN Schkopau GmbH': ['INOVYN Sales GmbH'],  
    'Metsä Tissue GmbH':['Metsä Tissue Immobilienverwaltungs GmbH'],
    'Mercer Stendal GmbH': ['Mercer Stendal Logistik GmbH'],
    'Mineraloelraffinerie Oberrhein GmbH & Co. KG': ['Mineraloelraffinerie Oberrhein Verwaltungs GmbH'],
    'OMV Deutschland GmbH': ['OMV Deutschland Services GmbH', 'OMV Deutschland Operations GmbH & Co. KG'],
    'Stahlwerk Annahütte Max Aicher GmbH & Co. KG': ['Stahlwerk Annahütte Beteiligung GmbH'],
    'Wepa Deutschland GmbH & Co. KG': ['Wepa Deutschland Verwaltungs-GmbH']
}

# Columns to sum
column_sum = ['Stammkapital', 'Grundkapital', 'Hafteinlage']

# Ensure columns to sum are numeric
df[column_sum] = df[column_sum].apply(pd.to_numeric, errors='coerce')

# Iterate through the company groups
for company_principal, company_sec in companies_merge.items():
    # Filter rows for the main and secondary companies
    df_principal = df[df['name'] == company_principal]
    df_sec = df[df['name'].isin(company_sec)]
    
    # If both principal and secondary companies exist
    if not df_principal.empty and not df_sec.empty:
        # Sum all values from secondary companies across the three columns
        sum_val_sec = df_sec[column_sum].sum(skipna=True).sum()
        
        # Identify the non-zero column in the primary company
        non_zero_col = df_principal[column_sum].iloc[0].idxmax()
        
        # Add the sum of the secondary companies to the non-zero column of the principal company
        df.loc[df['name'] == company_principal, non_zero_col] += sum_val_sec

        # Remove the rows of the secondary companies
        df = df[~df['name'].isin(company_sec)]

# Resulting DataFrame
df_h2b = df

#### Group by the columns 'name', 'register_identifier', 'zip', and 'CompanyName' Sum the 'H2-Potential in TWh' and aggregate other columns (taking the first occurrence)

In [ ]:
df_merged2 = df_h2b.groupby(['name', 'register_identifier', 'zip', 'CompanyName','Subsector_Name'], as_index=False).agg({
    'Stammkapital': 'first',
    'Grundkapital': 'first',
    'Hafteinlage': 'first',
    'location_lat': 'first',
    'location_long': 'first',
    'location_address': 'first',
    'registered_address': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'H2-Potential in TWh': 'sum',  # Summing the H2-Potential in TWh
    'Address': 'first'
})

len(df_merged2)

In [ ]:
# Ensure columns are treated as numbers
# distinguishing between sectors
df = df_merged2
df['Stammkapital'] = pd.to_numeric(df['Stammkapital'])
df['Grundkapital'] = pd.to_numeric(df['Grundkapital'])
df['Hafteinlage'] = pd.to_numeric(df['Hafteinlage'])

# Create a new column 'Capital'
df['Capital'] = df['Stammkapital'] + df['Grundkapital'] + df['Hafteinlage']

# Format 'Capital' column to have two decimal places
df['Capital'] = df['Capital'].apply(lambda x: "{:.2f}".format(x))
df_merged3 = df


##### Divide potential proportionally between the companies from Handelsregister associated to a company in the reference database

In [ ]:
# Convert relevant columns to numeric, handling errors by setting them to NaN
df_merged3['Capital'] = pd.to_numeric(df_merged3['Capital'], errors='coerce')
df_merged3['H2-Potential in TWh'] = pd.to_numeric(df_merged3['H2-Potential in TWh'], errors='coerce')


# Count the occurrences of each CompanyName in duplicates3
duplicates3 = df_merged3[df_merged3.duplicated(subset=['CompanyName','Subsector_Name'], keep=False)]
# Count the occurrences of each CompanyName and Subsector_Name combination in duplicates32
company_subsector_count = duplicates3.groupby(['CompanyName', 'Subsector_Name']).size().reset_index(name='Count')

# Define the function to proportionally distribute 'H2-Potential in TWh'
def proportionally_distribute_potential(group):
    total_capital = group['Capital'].sum()  # Total Capital for the group
    h2_potential = group['H2-Potential in TWh'].iloc[0]  
    company_name = group['CompanyName'].iloc[0]  # Get the CompanyName for this group
    subsector_name = group['Subsector_Name'].iloc[0]  # Get the Subsector_Name for this group
    
    # Find the count of this combination in company_subsector_count
    match = company_subsector_count[
        (company_subsector_count['CompanyName'] == company_name) &
        (company_subsector_count['Subsector_Name'] == subsector_name)
    ]
    
    if not match.empty:
        count_in_duplicates2 = match['Count'].values[0]
    else:
        count_in_duplicates2 = 2  # Default to 1 if combination is not found
    
    # Calculate the divisor based on the count in company_subsector_count
    divisor = count_in_duplicates2 * 0.5
    if divisor == 0:  # To avoid division by zero
        divisor = 1
    
    # Proportionally distribute the 'H2-Potential in TWh' based on 'Capital' and divide by the divisor
    distributed_potential = (h2_potential * (group['Capital'] / total_capital)) / divisor
    
    # Create a new column to avoid SettingWithCopyWarning
    group = group.copy()
    #group['Distributed H2-Potential in TWh'] = distributed_potential
    group['Distributed H2-Potential in TWh'] = distributed_potential.round(4)

    return group

# Apply the function, grouping by both 'CompanyName' and 'Subsector_Name'
df_divided_potential = df_merged3.groupby(['CompanyName', 'Subsector_Name'], group_keys=False).apply(proportionally_distribute_potential)


In [ ]:
df_ref = df_divided_potential.drop(columns=['CompanyName','Latitude','Longitude','Address','registered_address'])
df_ref = df_ref.rename(columns={'name': 'Name', 'register_identifier': 'Register_identifier','location_address': 'Address','location_long': 'Longitude','location_lat':'Latitude','zip': 'Postalcode'})


In [ ]:
conn = db.connect('mydatabase.db')

df_ref.to_sql('H2_final_subsectors_F', conn, if_exists='replace', index=False)

# Close the connection
conn.close()